In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests
from lxml import etree

def find_upmost_parent(element, parent_list):
    if element.tag in parent_list:
        res = element.tag
    elif element.getparent().tag in parent_list:
        res = element.getparent().tag
    else:
        res = find_upmost_parent(element.getparent(), parent_list)
    return res

def isrepresented_by_children(disease, disease_group_tree, disease_res_df, column_name='disease'):
    children_diseases = set([item.tag for item in disease_group_tree.getchildren()])
    affected_individual_num = disease_res_df['Disease_number_within_Drug_Use'][disease_res_df[column_name]==disease].values[0]
    children_affected_individual_num = 0
    for child_disease in children_diseases:
        if child_disease in disease_res_df[column_name].unique():
            children_affected_individual_num += disease_res_df['Disease_number_within_Drug_Use'][disease_res_df[column_name]==child_disease].values[0]
    if children_affected_individual_num >= affected_individual_num*0.8:
        print(f"{disease} is fully represnted by its children.")
        return True
    else:
        return False

In [8]:
# read from the xml file
disease_et = etree.parse("../data/anomalies_ontology.xml")
drug_et = etree.parse("../data/medication_atccodes_ontology.xml")
res_df = pd.read_csv('../results/allcombinations_atccodes_results.tsv', sep='\t')
res_othermedication = pd.read_csv('../results/allcombinations_othermedications_results.tsv', sep='\t')
print(res_df.shape, res_othermedication.shape)

(5939, 10) (1140, 10)


Filtering criteria:
1. More than 10 individuals' mother take this medication and have this anomaly
2. p-value (after FDR-BH correction) below 0.05

In [9]:
res_df = res_df[res_df['Disease_number_within_Drug_Use']>3]
res_df['corrected_p'] = multipletests(res_df['p'], method='fdr_bh')[1]
sig_res_df = res_df[res_df['corrected_p']<0.05]
print(sig_res_df.shape)

res_othermedication = res_othermedication[res_othermedication['Disease_number_within_Drug_Use']>10]
res_othermedication['corrected_p'] = multipletests(res_othermedication['p'], method='fdr_bh')[1]
sig_res_othermedication = res_othermedication[res_othermedication['corrected_p']<0.05]
print(sig_res_othermedication.shape)

sig_res_df.sort_values(by=['corrected_p']).to_csv('../results/significant_atccodes_results.tsv',
                                                                 sep='\t', index=False)
sig_res_othermedication.sort_values(by=['corrected_p']).to_csv('../results/significant_othermedications_results.tsv',
                                                                 sep='\t', index=False)

(4, 11)
(3, 11)


Pruning criteria:
1.  If the children objects have more than 80% individuals, then its parent is discarded

In [13]:
# also save for the results before multiple testing correction
disease_root = disease_et.getroot()
parent_list = [item.tag for item in disease_root.getchildren()]
res_df['upmost_disease_parent'] = [find_upmost_parent(disease_et.xpath(f'//{disease}')[0], parent_list) 
                                   for disease in res_df['disease']]
res_othermedication['upmost_disease_parent'] = [find_upmost_parent(disease_et.xpath(f'//{disease}')[0], parent_list) 
                                   for disease in res_othermedication['disease']]

drug_root = drug_et.getroot()
drug_parent_list = [item.tag for item in drug_root.getchildren()]
res_df['upmost_drug_parent'] = [find_upmost_parent(drug_et.xpath(f'//{drug}')[0], drug_parent_list) 
                                   for drug in res_df['drug']]

res_df_noduplicated_diseases = pd.DataFrame()
for disease_group in res_df['upmost_disease_parent'].unique():
    drug_list = res_df['drug'][(res_df['upmost_disease_parent']==disease_group)].unique()
    for drug in drug_list:
        subset_res_df = res_df[(res_df['upmost_disease_parent']==disease_group) & 
                                   (res_df['drug']==drug)]
        subset_res_df['is_represented_by_children_diseases'] = [isrepresented_by_children(disease, 
                                                                                          disease_et.xpath(f'//{disease}')[0], 
                                                                                          subset_res_df)
                                                                for disease in subset_res_df['disease']]

        num = subset_res_df[subset_res_df['is_represented_by_children_diseases']].shape[0]
        if num > 0:
            print(f"Deleting {num} diseases that are fully represented in its children.")
        subset_res_df = subset_res_df[subset_res_df['is_represented_by_children_diseases']==False]
        res_df_noduplicated_diseases = pd.concat([res_df_noduplicated_diseases, subset_res_df], axis=0)

res_othermedication_noduplicated_diseases = pd.DataFrame()
for disease_group in res_othermedication['upmost_disease_parent'].unique():
    drug_list = res_othermedication['drug'][(res_othermedication['upmost_disease_parent']==disease_group)].unique()
    for drug in drug_list:
        subset_res_df = res_othermedication[(res_othermedication['upmost_disease_parent']==disease_group) & 
                                                (res_othermedication['drug']==drug)]
        subset_res_df['is_represented_by_children_diseases'] = [isrepresented_by_children(disease, 
                                                                                          disease_et.xpath(f'//{disease}')[0], 
                                                                                          subset_res_df)
                                                                for disease in subset_res_df['disease']]

        num = subset_res_df[subset_res_df['is_represented_by_children_diseases']].shape[0]
        if num > 0:
            print(f"Deleting {num} diseases that are fully represented in its children.")
        subset_res_df = subset_res_df[subset_res_df['is_represented_by_children_diseases']==False]
        res_othermedication_noduplicated_diseases = pd.concat([res_othermedication_noduplicated_diseases, 
                                                                   subset_res_df], axis=0)
    

res_df_uniquecombi = pd.DataFrame()
for drug_group in res_df_noduplicated_diseases['upmost_drug_parent'].unique():
    for disease in res_df_noduplicated_diseases['disease'][res_df_noduplicated_diseases['upmost_drug_parent']==drug_group].unique():
        subset_res_df = res_df_noduplicated_diseases[(res_df_noduplicated_diseases['upmost_drug_parent']==drug_group) & 
                                                     (res_df_noduplicated_diseases['disease']==disease)]
        subset_res_df['is_represented_by_children_drugs'] = [isrepresented_by_children(drug, 
                                                                                       drug_et.xpath(f'//{drug}')[0], 
                                                                                       subset_res_df,
                                                                                       'drug')
                                                             for drug in subset_res_df['drug']]
        num = subset_res_df[subset_res_df['is_represented_by_children_drugs']].shape[0]
        if num > 0:
            print(f"Deleting {num} drug that are fully represented in its children.")
        res_df_uniquecombi = pd.concat([res_df_uniquecombi,
                                       subset_res_df[subset_res_df['is_represented_by_children_drugs']==False]],
                                       axis=0)

nervsyst is fully represnted by its children.
ntd is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
ntd is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
ntd is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
ntd is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
ntd is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
ntd is fully represnted by its children.
Deleting 1 diseases that are f

/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ntd is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
ntd is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
ntd is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
ntd is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
ntd is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
ntd is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
nervsyst is fully represnted by its children.
ntd is fully

heartdef is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
heartdef is fully represnted by its children.
alltranspos is fully represnted by its children.
VSD is fully represnted by its children.
Deleting 3 diseases that are fully represented in its children.
heartdef is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
heartdef is fully represnted by its children.
alltranspos is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
heartdef is fully represnted by its children.
VSD is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
heartdef is fully represnted by its children.
VSD is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
heartdef is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.
heartdef 

liporpal is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
ofclefts is fully represnted by its children.
liporpal is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
ofclefts is fully represnted by its children.
liporpal is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
ofclefts is fully represnted by its children.
liporpal is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
ofclefts is fully represnted by its children.
liporpal is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
ofclefts is fully represnted by its children.
liporpal is fully represnted by its children.
Deleting 2 diseases that are fully represented in its children.
ofclefts is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.


...

A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
A10T1 is fully represnted by its children.
Deleting 1 drug that are fully

Deleting 2 drug that are fully represented in its children.
B03T1 is fully represnted by its children.
B03BT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
B03T1 is fully represnted by its children.
B03BT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
B03T1 is fully represnted by its children.
B03BT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
B03T1 is fully represnted by its children.
B03BT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
B03T1 is fully represnted by its children.
B03BT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
B03T1 is fully represnted by its children.
B03BT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
B03T1 is fully represnted by its children.
B03BT1 is fully

J01T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
J01T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
J01T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
J01T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
J01T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N02T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N02T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N02T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N02T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N02T1 is fully represnted by its children.
Deleting 1 drug that are fully

N06AT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
N06T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N06T1 is fully represnted by its children.
N06AT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
N06T1 is fully represnted by its children.
N06AT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
N06T1 is fully represnted by its children.
N06AT1 is fully represnted by its children.
Deleting 2 drug that are fully represented in its children.
N06T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N06T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N06T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
N06T1 is fully represnted by its children.


R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully represented in its children.
R03T1 is fully represnted by its children.
Deleting 1 drug that are fully

In [14]:
res_df_uniquecombi.sort_values(by=['upmost_disease_parent', 
                                   'upmost_drug_parent', 'p']).to_csv('../results/pruned_atccodes_results_noMultiTestCorr.tsv',
                                                                 sep='\t', index=False)

res_othermedication_noduplicated_diseases.sort_values(by=['upmost_disease_parent', 'p']).to_csv('../results/pruned_othermedications_results_noMultiTestCorr.tsv',
                                                                 sep='\t', index=False)

In [17]:
res_df_uniquecombi.sort_values(by=['p']).head(10)

,Drug_use_number,Disease_number_within_Drug_Use,Disease_ratio_within_Drug_Use,p,expected_disease_ratio,odds_ratio,odds_ratio_ci_lower,odds_ratio_ci_higher,disease,drug,corrected_p,upmost_disease_parent,upmost_drug_parent,is_represented_by_children_diseases,is_represented_by_children_drugs
5790,54,9,0.166667,6.833670e-240,0.065587,2942.400000,168.036130,51522.953856,valproate,N03T1,1.790421e-236,terasyn,N03T1,False,False
184,54,7,0.129630,2.038212e-07,1.107692,7.406750,3.292029,16.664475,spinabif,N03T1,1.780038e-04,nervsyst,N03T1,False,False
17,12,5,0.416667,4.248974e-05,0.855061,9.389511,2.969868,29.685805,nervsyst,C09T1,2.783078e-02,nervsyst,C09T1,False,False
112,44,4,0.090909,1.725999e-04,0.605668,7.416327,2.603150,21.128975,anenceph,G02T1,9.044234e-02,nervsyst,G02T1,False,False
2255,54,5,0.092593,2.589251e-04,0.961943,5.808292,2.276306,14.820615,coarctat,N03T1,1.130640e-01,heartdef,N03T1,False,False
4719,27,4,0.148148,4.552088e-04,0.670445,6.959420,2.382287,20.330686,reddefec,N06AXT1,1.526504e-01,limbanom,N06T1,False,False
2836,12,4,0.333333,4.661081e-04,0.686640,8.307143,2.491425,27.698450,liporpal,D08T1,1.526504e-01,ofclefts,D08T1,False,False
3959,366,7,0.019126,7.019555e-04,1.926316,4.272632,1.873043,9.746380,bilrenage,B03AT1,1.797346e-01,urinary,B03T1,False,False
3209,241,12,0.049793,7.546110e-04,4.520783,2.905615,1.584503,5.328231,anorecta,R03AT1,1.797346e-01,digestiv,R03T1,False,False
4141,52,7,0.134615,1.046709e-03,1.985965,3.991465,1.783892,8.930920,hydronep,D10T1,1.844976e-01,urinary,D10T1,False,False


In [4]:
# add the most upper parent to each medication & anomaly
disease_root = disease_et.getroot()
parent_list = [item.tag for item in disease_root.getchildren()]
sig_res_df['upmost_disease_parent'] = [find_upmost_parent(disease_et.xpath(f'//{disease}')[0], parent_list) 
                                   for disease in sig_res_df['disease']]
sig_res_othermedication['upmost_disease_parent'] = [find_upmost_parent(disease_et.xpath(f'//{disease}')[0], parent_list) 
                                   for disease in sig_res_othermedication['disease']]

drug_root = drug_et.getroot()
drug_parent_list = [item.tag for item in drug_root.getchildren()]
sig_res_df['upmost_drug_parent'] = [find_upmost_parent(drug_et.xpath(f'//{drug}')[0], drug_parent_list) 
                                   for drug in sig_res_df['drug']]

/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [5]:
res_df_noduplicated_diseases = pd.DataFrame()
# disease_group = 'nervsyst'
for disease_group in sig_res_df['upmost_disease_parent'].unique():
    drug_list = sig_res_df['drug'][(sig_res_df['upmost_disease_parent']==disease_group)].unique()
    for drug in drug_list:
        subset_res_df = sig_res_df[(sig_res_df['upmost_disease_parent']==disease_group) & 
                                   (sig_res_df['drug']==drug)]
        subset_res_df['is_represented_by_children_diseases'] = [isrepresented_by_children(disease, 
                                                                                          disease_et.xpath(f'//{disease}')[0], 
                                                                                          subset_res_df)
                                                                for disease in subset_res_df['disease']]

        num = subset_res_df[subset_res_df['is_represented_by_children_diseases']].shape[0]
        if num > 0:
            print(f"Deleting {num} diseases that are fully represented in its children.")
        subset_res_df = subset_res_df[subset_res_df['is_represented_by_children_diseases']==False]
        res_df_noduplicated_diseases = pd.concat([res_df_noduplicated_diseases, subset_res_df], axis=0)

sig_res_othermedication_noduplicated_diseases = pd.DataFrame()
# disease_group = 'nervsyst'
for disease_group in sig_res_othermedication['upmost_disease_parent'].unique():
    drug_list = sig_res_othermedication['drug'][(sig_res_othermedication['upmost_disease_parent']==disease_group)].unique()
    for drug in drug_list:
        subset_res_df = sig_res_othermedication[(sig_res_othermedication['upmost_disease_parent']==disease_group) & 
                                                (sig_res_othermedication['drug']==drug)]
        subset_res_df['is_represented_by_children_diseases'] = [isrepresented_by_children(disease, 
                                                                                          disease_et.xpath(f'//{disease}')[0], 
                                                                                          subset_res_df)
                                                                for disease in subset_res_df['disease']]

        num = subset_res_df[subset_res_df['is_represented_by_children_diseases']].shape[0]
        if num > 0:
            print(f"Deleting {num} diseases that are fully represented in its children.")
        subset_res_df = subset_res_df[subset_res_df['is_represented_by_children_diseases']==False]
        sig_res_othermedication_noduplicated_diseases = pd.concat([sig_res_othermedication_noduplicated_diseases, 
                                                                   subset_res_df], axis=0)
    

res_df_uniquecombi = pd.DataFrame()
for drug_group in res_df_noduplicated_diseases['upmost_drug_parent'].unique():
    for disease in res_df_noduplicated_diseases['disease'][res_df_noduplicated_diseases['upmost_drug_parent']==drug_group].unique():
        subset_res_df = res_df_noduplicated_diseases[(res_df_noduplicated_diseases['upmost_drug_parent']==drug_group) & 
                                                     (res_df_noduplicated_diseases['disease']==disease)]
        subset_res_df['is_represented_by_children_drugs'] = [isrepresented_by_children(drug, 
                                                                                       drug_et.xpath(f'//{drug}')[0], 
                                                                                       subset_res_df,
                                                                                       'drug')
                                                             for drug in subset_res_df['drug']]
        num = subset_res_df[subset_res_df['is_represented_by_children_drugs']].shape[0]
        if num > 0:
            print(f"Deleting {num} drug that are fully represented in its children.")
        res_df_uniquecombi = pd.concat([res_df_uniquecombi,
                                       subset_res_df[subset_res_df['is_represented_by_children_drugs']==False]],
                                       axis=0)

terasyn is fully represnted by its children.
Deleting 1 diseases that are fully represented in its children.


/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/shuang/miniconda3/envs/methylation/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [6]:
# save pruned results
import os
if not os.path.isdir('../results'):
    os.mkdir('../results')
res_df_uniquecombi.sort_values(by=['upmost_disease_parent', 
                                   'upmost_drug_parent']).to_csv('../results/pruned_atccodes_results.tsv',
                                                                 sep='\t', index=False)

sig_res_othermedication_noduplicated_diseases.sort_values(by=['upmost_disease_parent']).to_csv('../results/pruned_othermedications_results.tsv',
                                                                 sep='\t', index=False)